In [2]:
!pip install torch
!pip install transformers

In [3]:
import pandas as pd
import os
import random
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE" # needed for making torch work?
import torch
from transformers import DistilBertTokenizer, DistilBertModel

In [3]:
# Read authentication keys from environmental variables
_open_ai_tkn = os.environ.get('OPENAI_KEY')
_project_tkn = os.environ.get('OPENAI_PROJECT')
_organisation_tkn = os.environ.get('OPENAI_ORG')

In [4]:
splits = {'train': 'training.csv', 'test': 'test.csv'}
df = pd.read_csv("hf://datasets/Annanay/aml_song_lyrics_balanced/" + splits["train"])

In [15]:
from openai import OpenAI

client = OpenAI(
  organization=_organisation_tkn,
  project=_project_tkn,
  api_key=_open_ai_tkn
)

In [1]:
def prompt_song(test_song):
    test_song_lines = test_song.split('\n')
    test_song_lines = [s for s in test_song_lines if len(s) > 0]
    old_lines = []
    new_lines = []
    for i in range(0, len(test_song_lines) - 7):
        question = "Please write a unique one-line lyric between these two stanzas of three lines: \n"
        group_1 = '\n'.join(test_song_lines[i:i+3])
        group_2 = '\n'.join(test_song_lines[i+4: i+7])
        prompt = f"{question}{group_1}\n\n{group_2}\n"
        #print(prompt)
        response = client.chat.completions.create(
            messages=[{
                "role": "user",
                "content": prompt
            }],
            model="gpt-4o-mini",
            max_completion_tokens=100
        )
        new_line = response.choices[0].message.content.split('\n')[0] # parsing the output
        old_lines.append(test_song_lines[i+3])
        new_lines.append(new_line)
    return new_lines, old_lines

In [10]:
test_song1 = 'In this green and pleasant land\nWe have a dream to understand\nIn the mountains of the mind\nThere is a spirit you will find\nJust like the angel from above\nSent to deliver words of love\nAncient cross and Zion star\nEastern ways and praise to Jah\n\nThis is our land\nThis is your land\nThis is our inheritance\nTo lead you on a merry dance\nIn the beginning there was light\nShining path and journeys bright\nWhen the nations pray as one\nThen will the races all be won\nIn this green and pleasant land\nWe have a dream to understand\nOpen paradises gate\nDance on the land and celebrate\n\nSay no matter what your colour\nYour race or your culture\nThis is our inheritance\nTo lead you on a merry dance'
test_song2 = "Love and happiness...\nSomething that can make you do wrong, make you do right...\nLove...\n\nLove and happiness\nWait a minute...\nSomething's going wrong\nSomeone's on the phone\nThree o'clock in the morning\nTalkin' about how she can make it right\nWell\nHappiness is when you really feel good with somebody\nNothing wrong with being in one with someone\nOh, baby, love and happiness\nLove and happiness...\nLove and happiness...\nLove and happiness\nYou be good to me\nI'll be good to you\nWe'll be together\nWe'll see each other\nWalk away with victory\nOh baby\nLove and happiness...\nLove and happiness...\n\nMake you do right... love'll make you do wrong...\nMake you come home early...\nMake you stay out all night long...\nThe power of love...\n\nWait a minute\nLet me tell you...\nThe power of love...\nMake you do right... love'll make you do wrong...\nMake you want to dance...\nLove and happiness...\nLove and happiness...\n\nLove is... wait a minute... love is...\nWalkin' together...\nTalkin' together...\nSay it again...\nSay it together...\nMmmm...."

new_lines, old_lines = prompt_song(test_song1)

NameError: name 'prompt_song' is not defined

In [ ]:
import torch
from torch.nn.functional import cosine_similarity

# Load pre-trained model tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def encode_sentences(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    return inputs

def get_embeddings(sentences):
    inputs = encode_sentences(sentences)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embeddings = outputs.last_hidden_state[:, 0, :]  # Get the [CLS] token embeddings
    return cls_embeddings

def compute_similarity(sentence1, sentence2):
    embeddings = get_embeddings([sentence1, sentence2])
    similarity = cosine_similarity(embeddings[0].unsqueeze(0), embeddings[1].unsqueeze(0))
    return similarity.item()

# Example sentences
print(tokenizer)
sentence1 = "I love playing football."
sentence2 = "I enjoy playing soccer."
similarity_score = compute_similarity(sentence1, sentence2)
print(f"Semantic Similarity Score: {similarity_score:.4f}")

In [ ]:
# compute semantic similarities of new and old sentences
similarities = []
for (s_new, s_old) in zip(new_lines, old_lines):
    similarities.append(compute_similarity(s_new, s_old))
similarities = np.array(similarities)
print(f'semantic similarity of {len(similarities)} generated sentences: \n mean: {np.mean(similarities)} \n std: {np.std(similarities)}')